In [62]:
import re
import pandas as pd
import emoji
import string
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
def clean_title(x):
    tag = r"([\s\(\)\[\],\/-]*\b((\d{1,2})[\s\(\)\[\],\/-]*([mf]|female|male))\b[\s\(\)\[\],\/-]*|\b[\s\(\)\[\],\/-]*(([mf]|female|male)[\s\(\)\[\],\/-]*(\d{1,2}))\b)"
    age = r"\d{1,2}"
    gender = r"(\b|\d)([mMfF])"
    match = re.sub(tag, '', x, flags=re.I)
    match = emoji.get_emoji_regexp().sub(r'', match)
    match = match.translate(str.maketrans('', '', string.punctuation))
    return match

In [114]:
df = pd.read_csv("../data/posts.csv", index_col="id")
ratings = df[["title", "average_rating"]].dropna(subset=["average_rating"])

X = ratings.title
X = X.apply(clean_title)
y = ((ratings.average_rating - ratings.average_rating.mean()) / ratings.average_rating.std())
y = y > 0

vectorizer = CountVectorizer(min_df=5, ngram_range=(3, 4))
X = vectorizer.fit_transform(X)
print(X.shape)

(27454, 6314)


In [111]:
from sklearn.linear_model import LinearRegression, Ridge

In [149]:
reg = LinearRegression().fit(X, y)
print(reg.score(X, y))
print("most negative")
print("\n".join([str((vectorizer.get_feature_names()[i], clf.coef_[0][i])) for i in reg.coef_.argsort()[:10]]))
print("most positive")
print("\n".join([str((vectorizer.get_feature_names()[i], clf.coef_[0][i])) for i in (-reg.coef_).argsort()[:10]]))


0.22176161837916128
most negative
('of what others', -0.7492272507705166)
('not having much luck', -0.6270846495553843)
('open to suggestions on', -0.4754578949958506)
('lost some weight recently', -0.470338076228774)
('would love to get', -0.662194797101952)
('should do to improve', -1.1037925933502166)
('lose weight and', -0.5680161132331454)
('to look my best', -1.172101077252358)
('for ways to improve', -1.2474713250966398)
('anything like this before', -0.9570684704121255)
most positive
('been going to the', 1.7097598896665214)
('change up my hair', 0.7371599912185377)
('curious of what others', 1.5474144762727766)
('idea what people', 0.827201184709977)
('of what others think', 0.47846469356326105)
('really know where stand', 1.2798309355590192)
('on what people', 0.7258458306982882)
('me what you got', 1.5799348980222172)
('im wondering if its', 1.732350265202244)
('be honest can handle', 1.2549448937541152)


In [144]:
ridge = Ridge().fit(X, y)
print(ridge.score(X, y))

print("most negative")
print("\n".join([str((vectorizer.get_feature_names()[i], clf.coef_[0][i])) for i in ridge.coef_.argsort()[:10]]))
print("most positive")
print("\n".join([str((vectorizer.get_feature_names()[i], clf.coef_[0][i])) for i in (-ridge.coef_).argsort()[:10]]))


0.1984468498058125
most negative
('comes to my', -2.092183201089716)
('its time to', -1.891713939882907)
('honestly think of', -1.7752841113160778)
('look better without', -1.8870579775831096)
('and still have', -1.4127107031415322)
('decided to post here', -1.4647043989078616)
('is the only', -1.4063154710217993)
('the process of losing', -1.5549577777066916)
('since last post', -1.5371954156819194)
('pull any punches', -1.5673035481774353)
most positive
('what you would rate', 1.7662446674834407)
('im wondering if its', 1.732350265202244)
('love to get some', 1.622035060122196)
('verification pic is', 1.7182537619908596)
('me also do', 1.4601354358488672)
('been going to the', 1.7097598896665214)
('people say look like', 1.3028238247098276)
('and im not sure', 1.7767214930038804)
('version of myself', 1.4391663698424726)
('see what others think', 1.2520925960280278)


In [143]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X, y)

print("most negative")
print("\n".join([str((vectorizer.get_feature_names()[i], clf.coef_[0][i])) for i in clf.coef_[0].argsort()[:10]]))
print("most positive")
print("\n".join([str((vectorizer.get_feature_names()[i], clf.coef_[0][i])) for i in (-clf.coef_[0]).argsort()[:10]]))

most negative
('comes to my', -2.092183201089716)
('its time to', -1.891713939882907)
('look better without', -1.8870579775831096)
('comment on my appearance', -1.8390731396446545)
('honestly think of', -1.7752841113160778)
('before want to', -1.6871411753876862)
('pull any punches', -1.5673035481774353)
('the process of losing', -1.5549577777066916)
('to cut my hair', -1.5398032301028568)
('since last post', -1.5371954156819194)
most positive
('and im not sure', 1.7767214930038804)
('what you would rate', 1.7662446674834407)
('im wondering if its', 1.732350265202244)
('verification pic is', 1.7182537619908596)
('been going to the', 1.7097598896665214)
('love to get some', 1.622035060122196)
('bad at taking', 1.610015552352932)
('me what you got', 1.5799348980222172)
('me to do', 1.575397186844696)
('me but be', 1.551266565263146)


In [148]:
from sklearn.metrics import *

print("Accuracy:", accuracy_score(y, clf.predict(X)))
print("AUROC:", roc_auc_score(y, clf.decision_function(X)))

Accuracy: 0.6911196911196911
AUROC: 0.7629669158689096


In [19]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [5]:
df = pd.read_csv("../data/posts.csv", index_col="id")

In [27]:
df.iloc[1].title

'[20M] What do you think of the Arabic /Russian mix ☺️?'

In [156]:
doc = nlp(clean_title(df.iloc[1].title))

In [158]:
displacy.render(doc)

In [66]:
frags

['do think', 'think', 'think of']

In [95]:

def extract_fragments(text):
    """
    Following the intuition that the
    bulk of this functional information is contained in
    the root of a question’s dependency parse along
    with its outgoing arcs (Iyyer et al., 2014a), we
    take the fragments of a question to be the root of
    its parse tree, along with each (root, child) pair.
    To capture cases when the operational word in
    the question is not connected to its root (such as
    “What...”), we also consider the initial unigram
    and bigram of a question as fragments. The following
    question has 5 fragments:
        what, what is, going→*, is←going and going→do
        
        What is the minister going to do about ... ?
    """
    doc = nlp(clean_title(text))
    if len(doc) == 0:
        return []
    fragments = [str(doc[0]), str(doc[:2])]
    # We take as NPs subtrees connected to the root with the
    # following: nsubj, nsubjpass, dobj, iobj, pobj, attr.

    np_deps = ["nsubj", "nsubjpass", "dobj", "iobj", "pobj", "attr"]
    root = None
    for tok in doc:

        if tok.head.dep_ == "ROOT" and tok.dep_ not in np_deps:
            if tok.dep_ == "ROOT":
                root = tok
                fragments.append(str(tok))
                continue
            if not root:
                fragments.append("{} {}".format(tok, tok.head))
                continue
            else:
                fragments.append("{} {}".format(tok.head, tok))
                continue
    return fragments

In [96]:
fragments = df.title.apply(extract_fragments)

In [97]:
fragments.head()

id
dh3ha2                                [How, Hows, How s, s]
dh3b61           [What, What do, do think, think, think of]
dh37sw    [Kinda, Kinda low, Kinda esteem, self esteem, ...
dh368y    [Got, Got a, Got are, are, are with, are some,...
dh32ch                             [RateMe, RateMe, RateMe]
Name: title, dtype: object

In [98]:
frag_list = []
for frags in fragments.to_numpy():
    frag_list += frags

In [106]:
frag_list = pd.Series(frag_list).str.strip().str.lower()

In [153]:
frag_list[frag_list.apply(lambda x: len(x.split()) >= 2)].value_counts()[:50]

rate me            8430
be honest          3735
just curious       2809
looking for        2688
please rate        1390
rate please        1155
please be          1146
do think           1083
rate and            911
give me             892
think do            889
do look             739
what do             693
honest opinions     640
just wondering      638
let know            626
rate my             603
always been         594
want know           577
just want           555
never had           521
just got            514
be please           511
m curious           492
how look            465
just looking        465
look do             460
curious about       429
curious think       428
am i                424
first time          416
not sure            410
how do              405
rate honestly       396
rate 110            380
i have              374
would like          356
would love          334
lost and            330
been told           330
’m curious          326
know do         